# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링 (목표 user 1000명)

- user_profile_df : User의 아이디, 리뷰수, 팔로워, url을 저장함
- user_df : '아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수' 를 저장함


### 개요 

1. 매장 정보 크롤링 결과에서 업태구분 별로 리뷰가 많은 식당 n개를 골라 가져온다
2. 리뷰가 많은 식당 n개에서 각각 3명씩 ( 추천순 or 최신순 일지는 아직 미정) 

In [22]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.action_chains import ActionChains

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  

from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import os
from datetime import datetime

import pandas as pd
from selenium.webdriver.common.by import By

### 함수 

In [47]:
# User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 
def user_profile(url) :

    global driver
    
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    user_element = {}

    # User ID 찾기
    user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
    user_element['아이디'] = user_id_element.text if user_id_element else None

    # User의 인기도 : 리뷰수, 팔로워 찾기 
    user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
    
    for element in user_popularity:
        em_tag = element.find('em')
        if em_tag:
            key = element.text.replace(em_tag.text, '').strip()
            value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
            user_element[key] = value
    
    # User URL
    user_element['url'] = url

    return user_element


# 일자, 요일, 방문일수 분류 함수 
def change_date_format(날짜) :
    #  부분 추출 (년월일까지)
    date_part = ""
    for i in 날짜 :
        date_part += i
        date_part += ','
    date_part = date_part.split('\n')[1]

    # 일자, 요일, 방문일수 분류하기 
    date_part = date_part.split(',')
    date = date_part[0]
    weekday = date[-3:]
    revisit = int(date_part[1].replace('번째 방문', ''))

    # 일자 타입 변경하기 
    date = '24' + date if date.startswith('년') else date # 만약 연도가 없는 경우 24를 붙이기
    match = re.findall(r'(\d+)년 (\d+)월 (\d+)일', date) # 타입 바꾸기 
    year, month, day = map(int, match[0])

    # 날짜 객체로 변환
    date_object = datetime(2000+year, month, day) 
    formatted_date = date_object.strftime('%Y-%m-%d')

    return formatted_date, weekday, revisit


# 유저 리뷰 가져오는 함수 
def find_user_data(page_down,driver) : 
    # 맨 처음 게시물 클릭 
    button = driver.find_element(By.CLASS_NAME, '_3P-5HQ')
    button.click()
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # data를 담을 빈 리스트 선언
    data_list = [] 


    # 페이지 스크롤 횟수만큼 반복
    for _ in range(page_down):

        # 대기시간 5초 
        wait = WebDriverWait(driver, 5)

        # 요소를 찾을 때 대기 조건 추가
        user_review_elements = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, '_27tH92'))
        )

        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN) 

        # 리뷰에서 정보 가져오기
        for reviews_elements in user_review_elements:
            i = 0 # 카테고리와 주소 구분할 때 사용
            restaurant_elements=reviews_elements.find_elements(By.CLASS_NAME, '_1QGRWW')
            
            if bool(restaurant_elements): # 정보가 없는 경우가 있음

                # 매장명 찾기
                restaurant_name = restaurant_elements[0].text  

        #<--------------------------------------------------------------------------------------------------->
                # 카테고리와 주소 찾기 
                category_location_elements = reviews_elements.find_elements(By.CLASS_NAME, '_2vBfgu')
                category_location_soup = BeautifulSoup(category_location_elements[i].get_attribute('outerHTML'), 'html.parser')
                
                # span 태그 안에 있는 텍스트 가져오기
                span_elements = category_location_soup.find_all('span', class_='wzFIfJ')
                category = span_elements[0].text if span_elements and len(span_elements) > 0 else '없음'
                location = span_elements[1].text if span_elements and len(span_elements) > 1 else '없음'
                i += 1
        #<--------------------------------------------------------------------------------------------------->
                # 리뷰 찾기 
                review_buttons = driver.find_elements_by_class_name('xHaT3')  # 리뷰 버튼 클릭 
                for button1 in review_buttons:
                    try:
                        button1.click()
                    except :
                        pass

                review_elements = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')
                try : 
                    review_text = reviews_elements.find_elements(By.CLASS_NAME,'z0t_8b')[0].text # 리뷰 본문          
                    sub_info=reviews_elements.find_elements(By.CLASS_NAME,'_1tkuel') #세부정보
                    if bool(sub_info):
                        sub_info=sub_info[0].text
                    else:
                        sub_info=None
                except :
                    review_text = '리뷰 없음'
                    sub_info = ''

        #<--------------------------------------------------------------------------------------------------->
                # 태그 찾기 
                # 일정 개수가 넘어가면 리뷰가 숨겨져 표시되므로 
                reactions_elements=reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트
                command=False
                for x in reactions_elements:
                    if x.get_attribute("role"): # 리액션 숨김 없애기
                        x.click()
                        command=True
                    else:
                        continue
                if command:
                    reactions_elements = reviews_elements.find_elements(By.CLASS_NAME, 'COw42b') # 리엑션 리스트 재탐색
                reactions=list(map(lambda x:x.text,reactions_elements))
                # 리스트를 벗김 
                reactions = str(reactions).replace('[', '').replace(']', '')

        #<--------------------------------------------------------------------------------------------------->
                # 방문 일자, 재방문 이력 찾기
                date_info = reviews_elements.find_element(By.CLASS_NAME, '_15xwjO .hol3Ic').find_elements(By.CLASS_NAME,'_3nNYBi')
                date=[x.text for x in date_info]
                day,weekday,revisit = change_date_format(date) # 함수 사용
            
                # 정보 추가
                data_dict = {
                        '아이디' : user_data['아이디'],
                        '리뷰' : user_data['리뷰'],
                        '팔로워' : user_data['팔로워'],
                        '매장명': restaurant_name,
                        '카테고리': category,
                        '주소' : location,
                        '리뷰 내용': review_text,
                        "세부정보" : sub_info,
                        '태그': reactions,
                        '방문일자': day,
                        '요일' : weekday,
                        '재방문횟수':revisit}
                data_list.append(data_dict)
        user_df = pd.DataFrame(data_list)
        user_df = user_df.drop_duplicates()
  
    return user_df




### 1. 그룹 데이터에서 검색어 사용( 정렬이 완료되었다는 가정 하에)

In [11]:
res_group_df = pd.read_excel('매장_groupby_test.xlsx')
res_group_df.drop('Unnamed: 1', axis=1, inplace=True)
res_group_df


,업태구분,식당이름,업태구분.1,주소,메뉴,가격,리뷰,검색어
0,PC방,제로백PC 매직엔 개포점,PC방,서울 강남구 개포로 223 지층,메뉴없음,가격없음,86,"개포로 223, 매직엔 PC방"
1,"곱창,막창,양",만득이곱창,"곱창,막창,양",서울 강남구 강남대로124길 36 1층 만득이곱창,야채곱창(1~2인분),"16,000원",321,"강남대로124길 36, 만득이곱창"
2,NaN,삼성원조양곱창,"곱창,막창,양",서울 강남구 학동로101길 7 승창빌딩 1층,특양,"30,000원",315,"학동로101길 7, 삼성양곱창"
3,김밥,생강김밥 본점,김밥,서울 강남구 남부순환로 2738 상일빌딩,문어김밥,"4,800원",2032,남부순환로 2738 생강김밥
4,돈가스,혼돈,돈가스,서울 강남구 언주로115길 15 101호,멘치카츠,"13,000원",114,"언주로115길 15, 혼돈"
5,돼지고기구이,길목 신관,돼지고기구이,서울 강남구 봉은사로 504 진양빌딩 1층,투뿔목살,"18,000원",54,"봉은사로 504, 길목 신관"
6,"백반,가정식",다미가정식백반,"백반,가정식",서울 강남구 개포로20길 16,가정식백반,"9,000원",53,개포로20길 16 다미
7,뷔페,더 키푸드,뷔페,서울 강남구 선릉로 823 6층,정찬,"10,000원",87,"선릉로 823, 더키푸드"
8,"순대,순댓국",계경순대국 수서역벤처빌점,"순대,순댓국",서울 강남구 밤고개로1길 10 수서현대벤쳐빌 지하 2층 202호,순대국,"8,000원",371,"밤고개로1길 10, 계경순대국수서현대벤쳐빌점"
9,양식,뢰벤돈까스,양식,서울 강남구 테헤란로 124 삼원타워 지하1층 B102호,등심돈까스,"9,000원",302,테헤란로 124 뢰벤 돈까스


### NaN값, '전처리 필요' 없애기 ( vscode 컨디션 안좋아서 크롤링 안된경우 있음)



In [48]:
res_group_df = res_group_df.loc[res_group_df['주소'] != '전처리필요'].dropna()
res_df = res_group_df.copy()
res_df = res_df.reset_index(drop=True) # 인덱스 새로하기
res_df = res_df[:3] # test용
res_df

,업태구분,식당이름,업태구분.1,주소,메뉴,가격,리뷰,검색어
0,PC방,제로백PC 매직엔 개포점,PC방,서울 강남구 개포로 223 지층,메뉴없음,가격없음,86,"개포로 223, 매직엔 PC방"
1,"곱창,막창,양",만득이곱창,"곱창,막창,양",서울 강남구 강남대로124길 36 1층 만득이곱창,야채곱창(1~2인분),"16,000원",321,"강남대로124길 36, 만득이곱창"
2,김밥,생강김밥 본점,김밥,서울 강남구 남부순환로 2738 상일빌딩,문어김밥,"4,800원",2032,남부순환로 2738 생강김밥


### 업태구분별로 3개씩 url 가져오기

In [49]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())


# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()


In [50]:
user_url_info_df = pd.DataFrame()
search_name_list = []
user_url_list = []
user_name_list = []

for i in tqdm(range(0, len(res_df)), desc='진행 중', position=0, leave=True):
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))
    time.sleep(3)

    try:
        if driver.find_elements(By.ID, 'entryIframe'):
            entryIframe = driver.find_element(By.ID, 'entryIframe')
            driver.switch_to.frame(entryIframe)
            time.sleep(3)

        # url이 나타날때까지 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

        # html 긁어오기
        html_code = driver.page_source
        soup = BeautifulSoup(html_code, 'html.parser')

        # 리뷰들이 포함된 <li> 태그들을 선택
        reviews = soup.select('ul.Uf1BQ li.qrzj_')

        for review in reviews:
            # 리뷰 페이지의 href 가져오기
            user_url = review.select_one('div.ozEbl a.iz4rh').get('href')
            user_name = review.select_one('div.rg88i').text

            ## 리스트 ㄱ
            search_name_list.append(name)
            user_url_list.append(user_url)
            user_name_list.append(user_name)

    except Exception as e:
        print(f"에러 메시지: {str(e)}")


user_url_info_df = pd.DataFrame({'검색어' : search_name_list,
                                 '아이디' : user_name_list,
                             'url' : user_url_list
                             })

    
user_url_info_df


진행 중:   0%|          | 0/3 [00:00<?, ?it/s]

진행 중: 100%|██████████| 3/3 [00:41<00:00, 13.91s/it]


,검색어,아이디,url
0,"강남대로124길 36, 만득이곱창",먹는게 남는거임,https://m.place.naver.com/my/5b6fcf82adca47e0b...
1,"강남대로124길 36, 만득이곱창",졍3117,https://m.place.naver.com/my/5bf6a41117ddc35c6...
2,"강남대로124길 36, 만득이곱창",jyo****,https://m.place.naver.com/my/5be833949ce82c68a...
3,남부순환로 2738 생강김밥,GT0,https://m.place.naver.com/my/5e5b7c428f87a842b...
4,남부순환로 2738 생강김밥,dongle25,https://m.place.naver.com/my/5bdc34ca2919a24c3...
5,남부순환로 2738 생강김밥,깅도리,https://m.place.naver.com/my/5e7191d68f87a842b...


### url에 들어가서 유저 정보 저장

In [51]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) 

# driver = webdriver.Chrome()

user_profile_df = pd.DataFrame(columns=['아이디', '리뷰', '팔로워', '팔로잉', 'url'])

for url in tqdm(user_url_info_df['url'], desc='진행 중', position=0, leave=True):

    # User의 아이디, 리뷰수, 팔로워를 가져오는 함수  
    user_data = user_profile(url)
    
    # 데이터프레임으로 만들기
    user_data_df = pd.DataFrame([user_data])
    user_profile_df = pd.concat([user_profile_df, user_data_df], ignore_index=True)
    
user_profile_df

진행 중: 100%|██████████| 6/6 [00:16<00:00,  2.82s/it]


,아이디,리뷰,팔로워,팔로잉,url
0,먹는게 남는거임,986,63,0,https://m.place.naver.com/my/5b6fcf82adca47e0b...
1,졍3117,118,5,0,https://m.place.naver.com/my/5bf6a41117ddc35c6...
2,jyo****,12,0,0,https://m.place.naver.com/my/5be833949ce82c68a...
3,GT0,1326,0,0,https://m.place.naver.com/my/5e5b7c428f87a842b...
4,dongle25,445,27,0,https://m.place.naver.com/my/5bdc34ca2919a24c3...
5,깅도리,72,18,5,https://m.place.naver.com/my/5e7191d68f87a842b...


### url에 들어가서 유저 리뷰 저장

In [52]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install()) 

# driver = webdriver.Chrome()

# 스크롤 횟수 
page_down = 3

# 데이터프레임 선언
total_user_df =  pd.DataFrame()
profile_df = pd.DataFrame(columns=['아이디', '리뷰', '팔로워', '팔로잉', 'url'])

# 반복해서 url 정보 가져오기 
for url in tqdm(user_url_info_df['url'], desc='진행 중', position=0, leave=True):
    try : 
        # User의 아이디, 리뷰수, 팔로워를 가져오는 함수  
        user_data = user_profile(url)
        
        # 데이터프레임으로 만들기
        user_data_df = pd.DataFrame([user_data])
        user_profile_df = pd.concat([user_profile_df, user_data_df], ignore_index=True)
        
        # User review 정보를 가져오는 함수
        user_df = find_user_data(page_down, driver)

        # User review 정보 저장 
        total_user_df = pd.concat([total_user_df, user_df], ignore_index=True)

    except Exception as e:
        print(f"에러 메시지: {str(e)}")
        continue

진행 중:  17%|█▋        | 1/6 [00:07<00:37,  7.58s/it]

에러 메시지: list index out of range


진행 중:  50%|█████     | 3/6 [00:37<00:36, 12.12s/it]

에러 메시지: invalid literal for int() with base 10: '21.8.25.수'


진행 중: 100%|██████████| 6/6 [01:32<00:00, 15.42s/it]


에러 해결 필요
진행 중:  17%|█▋        | 1/6 [00:07<00:37,  7.58s/it]
에러 메시지: list index out of range
진행 중:  50%|█████     | 3/6 [00:37<00:36, 12.12s/it]
에러 메시지: invalid literal for int() with base 10: '21.8.25.수'
진행 중:  83%|████████▎ | 5/6 [01:11<00:14, 14.77s/it]

내용더보기 해결안됨

In [53]:
user_profile_df

,아이디,리뷰,팔로워,팔로잉,url
0,먹는게 남는거임,986,63,0,https://m.place.naver.com/my/5b6fcf82adca47e0b...
1,졍3117,118,5,0,https://m.place.naver.com/my/5bf6a41117ddc35c6...
2,jyo****,12,0,0,https://m.place.naver.com/my/5be833949ce82c68a...
3,GT0,1326,0,0,https://m.place.naver.com/my/5e5b7c428f87a842b...
4,dongle25,445,27,0,https://m.place.naver.com/my/5bdc34ca2919a24c3...
5,깅도리,72,18,5,https://m.place.naver.com/my/5e7191d68f87a842b...
6,먹는게 남는거임,986,63,0,https://m.place.naver.com/my/5b6fcf82adca47e0b...
7,졍3117,118,5,0,https://m.place.naver.com/my/5bf6a41117ddc35c6...
8,jyo****,12,0,0,https://m.place.naver.com/my/5be833949ce82c68a...
9,GT0,1326,0,0,https://m.place.naver.com/my/5e5b7c428f87a842b...


In [54]:
total_user_df

,아이디,리뷰,팔로워,매장명,카테고리,주소,리뷰 내용,세부정보,태그,방문일자,요일,재방문횟수
0,졍3117,118,5,소코아 잠실점,카레,서울특별시 송파구 송파동,매장이 청결하고 음식이 맛있어요,이용 방법예약 없이대기 시간바로 입장동행친구,"'음식이 맛있어요', '재료가 신선해요', '특별한 메뉴가 있어요', '가성비가 좋...",2024-03-03,일요일,1
1,졍3117,118,5,성수향수공방 뤼미에르퍼퓸 성수,공방,서울특별시 성동구 성수동2가,친절하시고 설명도 자세해서 처음 만드는 데도 어려움없이 잘 만들었어요 이색 데이트로...,None,"'원데이 클래스가 알차요', '추천을 잘해줘요', '설명이 자세해요', '친절해요'...",2024-02-28,수요일,1
2,졍3117,118,5,네일오브윤 분당점,"네일아트,네일샵",경기도 성남시 분당구 야탑동,봄네일아트하러 방문했는데 기대 이상으로 만족하고 나온 곳이에요! 친절하시고 네일 디...,None,"'디자인 추천을 잘해줘요', '디자인이 다양해요', '유지력이 좋아요', '원하는 ...",2024-02-23,금요일,1
3,졍3117,118,5,동묘마케트,와인,서울특별시 종로구 창신동,감바스 폭찹 떡볶이 모두 양도 많고 맛있어요ㅜㅜ가성비와 맛과 분위기까지 한번에 사로...,"이용 방법예약 후대기 시간바로 입장목적친목, 기념일동행친구","'음식이 맛있어요', '가성비가 좋아요', '특별한 메뉴가 있어요', '술이 다양해...",2024-02-22,목요일,1
4,졍3117,118,5,왕초바베큐,"치킨,닭강정",서울특별시 성동구 홍익동,"왕십리에 숯불바베큐 맛집이 생기다뇨,,너무 맛있어요\n기본으로 떡 사리는 들어있으니...",이용 방법예약 후대기 시간바로 입장목적친목동행친구,"'음식이 맛있어요', '재료가 신선해요', '양이 많아요', '가성비가 좋아요', ...",2024-02-20,화요일,1
...,...,...,...,...,...,...,...,...,...,...,...,...
69,깅도리,72,18,사계솔,"족발,보쌈",서울특별시 강남구 도곡동,찌개맛집이에요\n점심때마다 와서 먹는데 넘 맛있어요\n밥 리필가능!,,"'음식이 맛있어요', '양이 많아요', '가성비가 좋아요', '건강한 맛이에요'",2023-10-10,화요일,1
70,깅도리,72,18,냥냥탕후루,"카페,디저트",서울특별시 송파구 송파동,샤머를 좋아하는 1인. 설탕옷이 조금 두꺼워서 아쉽다 뭔가 얇아서 바사삭하는 느낌을...,,"'디저트가 맛있어요', '특별한 메뉴가 있어요', '가성비가 좋아요'",2023-10-08,일요일,1
71,깅도리,72,18,생생장어,"장어,먹장어요리",서울특별시 송파구 석촌동,가성비 좋아요! 야들야들하니 맛있습니다 친구 추천으로 왔는데 맛있게 잘먹었어요 소자...,,"'음식이 맛있어요', '재료가 신선해요', '가성비가 좋아요', '반찬이 잘 나와요'",2023-10-06,금요일,1
72,깅도리,72,18,본죽&비빔밥 매봉역점,죽,서울특별시 강남구 도곡동,양이 진짜 많아요! 반은 먹고 반은 포장해 갔어요 반찬이 진짜 맛있습니당 ㅋㅋ ㅋㅋ...,,"'음식이 맛있어요', '재료가 신선해요', '건강한 맛이에요', '반찬이 잘 나와요'",2023-09-26,화요일,1


In [55]:
# 유저의 리뷰
total_user_df.to_excel('data/user_review_data/user_df_test.xlsx', index=False)

# 유저의 정보 
user_profile_df.to_excel('data/user_review_data/user_profile_df_test.xlsx', index=False)